In [ ]:

#SELECT marketRegionId, marketMarketId, marketCollected, category1, category2, marketBrandGoldenApple, marketName, marketDiscountPrice, marketPriceValue, marketUrl,marketAvailability FROM pricing.competitors_price_sephora;

In [1]:
import numpy as np
import math
import pandas as pd

df = pd.read_csv(r'C:\Users\stukova_a\Downloads\competitors_price_sephora_202412311505.txt', 
                delimiter='\t',
                encoding='utf-8')

#pip install pandas openpyxl
# Находим колонки, которые содержат '|'
for column in df.columns:
    if df[column].dtype == 'object':  # Проверяем только текстовые колонки
        if df[column].str.contains('|', regex=False).any():
            # Разбиваем колонку на несколько новых
            split_cols = df[column].str.split('|', expand=True)
            
            # Называем новые колонки
            for i in range(len(split_cols.columns)):
                new_col_name = f"{column}_{i+1}"
                df[new_col_name] = split_cols[i]
            
            # Удаляем исходную колонку
            df = df.drop(columns=[column])
# Словарь для переименования колонок
df.columns  = ["Пусто1",
    "marketRegionId",
    "marketMarketId",
    "marketCollected",
    "category1",
    "category2",
    "marketBrandGoldenApple",
    "marketName",
    "marketDiscountPrice",
    "marketPriceValue",
    "marketUrl",
    "marketAvailability","Пусто2","Пусто3","Пусто4"]

#df
def remove_empty_columns(df):
    # Remove columns where all values are empty/null
    # This includes columns with only None, NaN, empty strings, or "Пусто"
    return df.dropna(axis=1, how='all').drop(columns=df.columns[
        (df == "Пусто1").all() | 
        (df == "Пусто2").all() | 
        (df == "Пусто3").all() | 
        (df == "Пусто4").all() | 

        (df == "").all() | 
        (df.isna().all())
    ])
df = df.drop(0)
def clean_columns(df):
    # Удаляем колонки, в названии которых есть "Пусто"
    clean_cols = [col for col in df.columns if not col.startswith('Пусто1')]
    clean_cols = [col for col in df.columns if not col.startswith('Пусто2')]
    clean_cols = [col for col in df.columns if not col.startswith('Пусто3')]
    clean_cols = [col for col in df.columns if not col.startswith('Пусто4')]

    # Из оставшихся колонок оставляем только те, где есть данные
    df = df[clean_cols]
    df = df.dropna(axis=1, how='all')
    
    return df

def clean_columns(df):
    # Удаляем колонки, которые:
    # 1. Содержат "Пусто" где угодно в названии
    # 2. Начинаются с цифр
    # 3. Состоят только из пустых значений
    
    clean_cols = [col for col in df.columns 
                 if 'Пусто' not in col  # убираем колонки с "Пусто"
                 and not any(c.isdigit() for c in col[0])  # убираем колонки, начинающиеся с цифр
                 and not df[col].isna().all()  # убираем полностью пустые колонки
                 and not (df[col] == '').all()  # убираем колонки с пустыми строками
                 and not (df[col] == 'None').all()]  # убираем колонки со значением None
    
    return df[clean_cols]
needed_columns = [
    'marketRegionId',
    'marketMarketId',
    'marketCollected',
    'category1',
    'category2',
    'marketBrandGoldenApple',
    'marketName',
    'marketDiscountPrice',
    'marketPriceValue',
    'marketUrl',
    'marketAvailability'
]

# Создаем новый датафрейм только с нужными колонками
new_df = df[needed_columns]

df = new_df

# Конвертируем ценовые колонки в числовой формат
df['marketDiscountPrice'] = pd.to_numeric(df['marketDiscountPrice'], errors='coerce')
df['marketPriceValue'] = pd.to_numeric(df['marketPriceValue'], errors='coerce')

# Расчет скидки
def calculate_discount(row):
    if pd.isna(row['marketDiscountPrice']) or pd.isna(row['marketPriceValue']):
        return None
    if row['marketPriceValue'] == 0:  # избегаем деления на ноль
        return 0

    # Определяем большую и меньшую цену
    base_price = max(row['marketPriceValue'], row['marketDiscountPrice'])
    discounted_price = min(row['marketPriceValue'], row['marketDiscountPrice'])

    discount = round(((base_price - discounted_price) / base_price) * 100, 2)

    # Если скидка 100%, значит данные некорректные
    if discount == 100:
        return 0
    return discount

# Добавляем колонку со скидкой и исправляем цены
df['discount_percent'] = df.apply(calculate_discount, axis=1)

# Где скидка была 100%, приравниваем цену после скидки к цене до скидки
mask = (df['discount_percent'] == 0) & (df['marketDiscountPrice'] != df['marketPriceValue'])
df.loc[mask, 'marketDiscountPrice'] = df.loc[mask, 'marketPriceValue']

# Проверяем среднюю скидку на наличие NaN и заменяем на 0, если они есть
average_discount = df['discount_percent'].mean()
if pd.isna(average_discount):
    average_discount = 0

# Вывод исправленных результатов
print("\nПримеры расчета скидок:")
print(df[['marketName', 'marketPriceValue', 'marketDiscountPrice', 'discount_percent']].head(10))

print("\nСтатистика по скидкам:")
print(f"Количество товаров со скидкой: {len(df[df['discount_percent'] > 0])}")
print(f"Средняя скидка: {average_discount:.2f}%")
print(f"Максимальная скидка: {df['discount_percent'].max():.2f}%")

# Проверка на 100% скидки
print("\nПроверка на наличие 100% скидок:")
hundred_percent = df[df['discount_percent'] == 100]
print(f"Количество товаров со 100% скидкой: {len(hundred_percent)}")

df['discount_percent'] = df['discount_percent'].replace([np.nan, None, 'NaN', 'None'], 0.0)


# Замена некорректных значений в discount_percent
df['discount_percent'] = df['discount_percent'].replace([np.nan, None, 'NaN', 'None'], 0.0)

# Очистка marketCollected перед преобразованием
# Удаляем лишние пробелы, заменяем некорректные значения на NaT
df['marketCollected'] = df['marketCollected'].str.strip()  # Удаляем лишние пробелы
df['marketCollected'] = pd.to_datetime(
    df['marketCollected'], errors='coerce', format='%Y-%m-%d %H:%M:%S'
)  # Преобразуем в дату, некорректные значения заменяются на NaT

# Удаляем время, оставляя только дату
df['marketCollected'] = df['marketCollected'].dt.date

# Проверка результата
print(df['marketCollected'].head())

# Удаляем пробелы и специальные символы
def clean_string(x):
    if isinstance(x, str):
        return x.strip().replace('¶', ' ')
    return x

# Применяем очистку только к строковым колонкам
df = df.apply(lambda x: x.map(clean_string) if x.dtype == "object" else x)


unique_columns = [
    "marketRegionId",
    "marketName",
    
    "category1",
    "marketCollected", "marketUrl"
]
df_unique = df.drop_duplicates(subset=unique_columns)

#df_unique

df = df_unique[~df_unique['marketAvailability'].str.contains('\d', regex=True, na=False)]
#df.head(10000).to_excel(r'C:\Users\stukova_a\Downloads\first_10_rows.xlsx', index=False)
#df

# Преобразуем marketCollected в datetime, если это ещё не сделано
df['marketCollected'] = pd.to_datetime(df['marketCollected'])

# Добавляем столбец с неделей года
df['week_of_year'] = df['marketCollected'].dt.isocalendar().week

# Добавляем столбец с названием месяца
df['month_name'] = df['marketCollected'].dt.strftime('%B')




Примеры расчета скидок:
                                           marketName  marketPriceValue  \
1   The Scent Elixir - Parfum Intense - 50ml      ...             470.0   
2   The Scent Elixir - Parfum Intense - 100ml     ...             650.0   
3   Jazz Club - Eau De Toilette - 30ml            ...             334.0   
4   Jazz Club - Eau De Toilette - Eau de Toilette ...             777.0   
5   Replica From The Garden - 100ml               ...             777.0   
6   Replica From The Garden - 30ml                ...             334.0   
7   The Scent Elixir - Parfum Intense - 50ml      ...             635.0   
8   Libre - Eau de Parfum - 30ml                  ...             426.0   
9   Libre - Eau de Parfum - 50ml                  ...             587.0   
10  Libre - Eau de Parfum - 90ml                  ...             777.0   

    marketDiscountPrice  discount_percent  
1                 470.0               0.0  
2                 650.0               0.0  
3                

C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_20496\1844157606.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['marketCollected'] = pd.to_datetime(df['marketCollected'])
C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_20496\1844157606.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['week_of_year'] = df['marketCollected'].dt.isocalendar().week
C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_20496\1844157606.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [ ]:
#df

In [ ]:
# Create availability summary table
def create_availability_summary(df):
    # Ensure proper data types
    df['marketCollected'] = pd.to_datetime(df['marketCollected'])
    
    # Create separate dataframes for InStock and OutOfStock
    df_instock = df[df['marketAvailability'].str.contains('InStock', case=False, na=False)]
    df_outstock = df[df['marketAvailability'].str.contains('OutOfStock', case=False, na=
    # Function to calculate metrics for each availability status
    def calculate_metrics(data):
        # Count total items
        total_count = len(data)
        
        # Calculate average discount only for items with discount > 0
        items_with_discount = data[data['discount_percent'] > 0]
        avg_discount = items_with_discount['discount_percent'].mean()
        discount_count = len(items_with_discount)
        
        return total_count, avg_discount, discount_count
    
    # Group by date and region for both InStock and OutOfStock
    results = []
    
    for date, region in df.groupby(['marketCollected', 'marketRegionId']):
        instock_data = df_instock[(df_instock['marketCollected'] == date[0]) & 
                                (df_instock['marketRegionId'] == date[1])]
        outstock_data = df_outstock[(df_outstock['marketCollected'] == date[0]) & 
                                  (df_outstock['marketRegionId'] == date[1])]
        
        instock_count, instock_avg_disc, instock_disc_count = calculate_metrics(instock_data)
        outstock_count, outstock_avg_disc, outstock_disc_count = calculate_metrics(outstock_data)
        
        results.append({
            'marketRegionId': date[1],
            'marketCollected': date[0],
            'week_of_year': date[0].isocalendar().week,
            'month_name': date[0].strftime('%B'),
            'InStocks_Count': instock_count,
            'InStocks_Avg_Discount': round(instock_avg_disc if pd.notnull(instock_avg_disc) else 0),
            'InStocks_Discount_Count': instock_disc_count,
            'OutOfStocks_Count': outstock_count,
            'OutOfStocks_Avg_Discount': round(outstock_avg_disc if pd.notnull(outstock_avg_disc) else 0),
            'OutOfStocks_Discount_Count': outstock_disc_count
        })
    
    result_df = pd.DataFrame(results)
    
    # Format the date
    result_df['marketCollected'] = result_df['marketCollected'].dt.strftime('%d.%m.%Y')
    
    # Sort by region and date
    result_df = result_df.sort_values(['marketRegionId', 'marketCollected'])
    
    # Rename columns to match desired format
    result_df.columns = [
        'marketRegionId', 'marketCollected', 'week_of_year', 'month_name',
        'InStocks', 'InStocks Avg_Discount', 'InStocks Discount_Count',
        'OutOfStocks', 'OutOfStocks Avg_Discount', 'OutOfStocks Discount_Count'
    ]
    
    return result_df

# Apply the function to your dataframe
result_df = create_availability_summary(df)

# Save to Excel if needed
result_df.to_excel('availability_summary.xlsx', index=False)

# Display the first few rows
print(result_df.head())

In [ ]:
#result_df.to_excel(r'C:\Users\stukova_a\Downloads\summary_report_SephoraTest21.xlsx', engine='openpyxl') 

In [ ]:
# Create availability summary table with categories
def create_availability_summary(df):
    # Ensure proper data types
    df['marketCollected'] = pd.to_datetime(df['marketCollected'])
    
    # Create separate dataframes for InStock and OutOfStock
    df_instock = df[df['marketAvailability'].str.contains('InStock', case=False, na=False)]
    df_outstock = df[df['marketAvailability'].str.contains('OutOfStock', case=False, na=False)]
    
    # Function to calculate metrics for each availability status
    def calculate_metrics(data):b 
        # Count total items
        total_count = len(data)
        
        # Calculate average discount only for items with discount > 0
        items_with_discount = data[data['discount_percent'] > 0]
        avg_discount = items_with_discount['discount_percent'].mean()
        discount_count = len(items_with_discount)
        
        return total_count, avg_discount, discount_count
    
    # Group by date, region, and category for both InStock and OutOfStock
    results = []
    
    for (date, region, category), group in df.groupby(['marketCollected', 'marketRegionId', 'category1']):
        instock_data = df_instock[
            (df_instock['marketCollected'] == date) & 
            (df_instock['marketRegionId'] == region) &
            (df_instock['category1'] == category)
        ]
        outstock_data = df_outstock[
            (df_outstock['marketCollected'] == date) & 
            (df_outstock['marketRegionId'] == region) &
            (df_outstock['category1'] == category)
        ]
        
        instock_count, instock_avg_disc, instock_disc_count = calculate_metrics(instock_data)
        outstock_count, outstock_avg_disc, outstock_disc_count = calculate_metrics(outstock_data)
        
        results.append({
            'marketRegionId': region,
            'marketCollected': date,
            'week_of_year': date.isocalendar().week,
            'month_name': date.strftime('%B'),
            'category1': category,
            'InStocks': instock_count,
            'InStocks_Avg_Discount': round(instock_avg_disc if pd.notnull(instock_avg_disc) else 0),
            'InStocks_Discount_Count': instock_disc_count,
            'OutOfStocks': outstock_count,
            'OutOfStocks_Avg_Discount': round(outstock_avg_disc if pd.notnull(outstock_avg_disc) else 0),
            'OutOfStocks_Discount_Count': outstock_disc_count
        })
    
    result_df2 = pd.DataFrame(results)
    
    # Format the date
    result_df2['marketCollected'] = result_df2['marketCollected'].dt.strftime('%d.%m.%Y')
    
    # Sort by region, date and category
    result_df2 = result_df2.sort_values(['marketRegionId', 'marketCollected', 'category1'])
    
    # Rename columns to match desired format
    result_df2.columns = [
        'marketRegionId', 'marketCollected', 'week_of_year', 'month_name', 'category1',
        'InStocks', 'InStocks Avg_Discount', 'InStocks Discount_Count',
        'OutOfStocks', 'OutOfStocks Avg_Discount', 'OutOfStocks Discount_Count'
    ]
    
    return result_df2

# Apply the function to your dataframe
result_df2 = create_availability_summary(df)

# Save to Excel if needed
result_df2.to_excel('availability_summary_by_category.xlsx', index=False)

# Display the first few rows
print(result_df2.head())

C:\Users\Stukova_a\AppData\Local\Temp\ipykernel_20496\2684954158.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['marketCollected'] = pd.to_datetime(df['marketCollected'])


In [ ]:
result_df2.to_excel(r'C:\Users\stukova_a\Downloads\summary_report_SephoraTest22.xlsx', engine='openpyxl') 

In [ ]:
# Определяем список уникальных столбцов
unique_columns = [
   "marketRegionId",
   "marketName",
   "category1",
   "week_of_year", 
   "marketUrl"
]

# Удаляем дубликаты
df_unique = df.drop_duplicates(subset=unique_columns)

df_unique.loc[df_unique['discount_percent'] < 5, 'discount_percent'] = 0


df = df_unique

# Оставляем только нужные колонки и фильтруем по InStocks
df_unique = df_unique[
   ['marketRegionId', 'category1', 'marketBrandGoldenApple', 'marketName', 'marketUrl',
    'marketAvailability', 'discount_percent', 'week_of_year', 'month_name']
]
df_unique = df_unique[df_unique['marketAvailability'] == 'InStocks']

# Разделяем на 3 датафрейма по регионам
dubai_df = df_unique[df_unique['marketRegionId'] == 'Дубай']
qatar_df = df_unique[df_unique['marketRegionId'] == 'Катар']
saudi_df = df_unique[df_unique['marketRegionId'] == 'Саудовская Аравия']

discount_mask = df['discount_percent'] > 0


In [ ]:


# Для листа Last - находим последний день с товарами > 27000
daily_counts = df.groupby(['marketCollected'])['marketName'].count().reset_index()
last_valid_date = daily_counts[daily_counts['marketName'] > 27000]['marketCollected'].max()
print(f"Последняя дата с более чем 27000 товаров: {last_valid_date}")

# Фильтруем данные за этот день
last_data = df[df['marketCollected'] == last_valid_date]

# Убираем дубликаты для Last
last_data_unique = last_data.drop_duplicates(subset=[
    'marketRegionId', 
    'marketName',
    'marketDiscountPrice',
    'marketPriceValue',
    'marketUrl'
])

# Предварительная обработка данных - удаление дублей по неделям с приоритетом по скидке
df_cleaned = df.sort_values(['marketRegionId', 'week_of_year', 'discount_percent'], ascending=[True, True, False])
df_cleaned = df_cleaned.groupby(['marketRegionId', 'marketName', 'week_of_year']).first().reset_index()

# Для листа Pivot
df_pivot = df_cleaned.copy()
summary = df_pivot.groupby(['marketRegionId', 'week_of_year']).agg(
    discounted_items=('discount_percent', lambda x: (x > 0).sum()),
    avg_discount=('discount_percent', lambda x: x[x > 0].mean() if (x > 0).any() else 0),
    in_stock=('marketAvailability', lambda x: (x == 'InStocks').sum()),
    out_of_stock=('marketAvailability', lambda x: (x == 'OutOfStocks').sum())
).reset_index()

# Добавляем столбец "Всего ассортимент" для Pivot
summary['total_items'] = (
    summary['discounted_items'] +
    summary['in_stock'] +
    summary['out_of_stock']
)

# Для страниц по регионам
dubai_df = df_cleaned[df_cleaned['marketRegionId'] == 'Дубай']
qatar_df = df_cleaned[df_cleaned['marketRegionId'] == 'Катар']
saudi_df = df_cleaned[df_cleaned['marketRegionId'] == 'Саудовская Аравия']

# Собираем все регионы обратно в один общий DataFrame
all_regions_df = pd.concat([dubai_df, qatar_df, saudi_df])

# Создание листа Category из общего DataFrame
summary_by_category = all_regions_df.groupby(['marketRegionId', 'week_of_year',  'category1']).agg(
    discounted_items=('discount_percent', lambda x: (x > 0).sum()),
    avg_discount=('discount_percent', lambda x: x[x > 0].mean() if (x > 0).any() else 0),
    in_stock=('marketAvailability', lambda x: (x == 'InStocks').sum()),
    out_of_stock=('marketAvailability', lambda x: (x == 'OutOfStocks').sum())
).reset_index()

# Добавляем столбец "Всего ассортимент" для Category
summary_by_category['total_items'] = (
    summary_by_category['discounted_items'] +
    summary_by_category['in_stock'] +
    summary_by_category['out_of_stock']
)

# Округляем скидки вверх
last_data_unique['discount_percent'] = last_data_unique['discount_percent'].apply(lambda x: math.ceil(x))
summary['avg_discount'] = summary['avg_discount'].apply(lambda x: math.ceil(x))
summary_by_category['avg_discount'] = summary_by_category['avg_discount'].apply(lambda x: math.ceil(x))

# Фильтруем по количеству товаров в наличии до переименования колонок
summary = summary[summary['in_stock'] >= 6000]

# Переименовываем колонки
summary = summary.rename(columns={
    'marketRegionId': 'Регион',
    'week_of_year': 'Неделя года',
    'month_name': 'Месяц',
    'discounted_items': 'Количество товаров со скидкой',
    'avg_discount': 'Средний размер скидки',
    'in_stock': 'В наличии (всего ассортимент)',
    'out_of_stock': 'Нет в наличии (всего ассортимент)',
    'total_items': 'Всего ассортимент'
})

summary_by_category = summary_by_category.rename(columns={
    'marketRegionId': 'Регион',
    'week_of_year': 'Неделя года',
    'month_name': 'Месяц',
    'category1': 'Категория',
    'discounted_items': 'Количество товаров со скидкой',
    'avg_discount': 'Средний размер скидки',
    'in_stock': 'В наличии (всего ассортимент)',
    'out_of_stock': 'Нет в наличии (всего ассортимент)',
    'total_items': 'Всего ассортимент'
})



In [ ]:
# Сохраняем результаты
with pd.ExcelWriter(r'C:\Users\stukova_a\Downloads\summary_report_SephoraTest17.xlsx', engine='openpyxl') as writer:
     dubai_df.to_excel(writer, sheet_name='Dubai', index=False)
     qatar_df.to_excel(writer, sheet_name='Qatar', index=False)
     saudi_df.to_excel(writer, sheet_name='Saudi', index=False)
     summary.to_excel(writer, sheet_name='Pivot', index=False)
     summary_by_category.to_excel(writer, sheet_name='Category', index=False)
     last_data_unique.to_excel(writer, sheet_name='Last', index=False)


In [ ]:
# Сохраняем результаты
with pd.ExcelWriter(r'C:\Users\stukova_a\Downloads\summary_report_SephoraTest18.xlsx', engine='openpyxl') as writer:
    # Для Dubai, Qatar, Saudi нужно только добавить Year, так как month_name уже есть
    dubai_df['Year'] = pd.to_datetime(dubai_df['marketCollected']).dt.year
    dubai_df.to_excel(writer, sheet_name='Dubai', index=False)

    qatar_df['Year'] = pd.to_datetime(qatar_df['marketCollected']).dt.year
    qatar_df.to_excel(writer, sheet_name='Qatar', index=False)

    saudi_df['Year'] = pd.to_datetime(saudi_df['marketCollected']).dt.year
    saudi_df.to_excel(writer, sheet_name='Saudi', index=False)

    # Для summary и summary_by_category используем 'Неделя года'
    def get_month_from_week(week_num):
        # Получаем первый день года
        year = 2024  # Используем текущий год
        first_day = pd.Timestamp(f'{year}-01-01')
        # Получаем начало недели
        start_date = first_day + pd.Timedelta(weeks=int(week_num)-1)
        # Получаем конец недели
        end_date = start_date + pd.Timedelta(days=6)
        
        start_month = start_date.strftime('%B')
        end_month = end_date.strftime('%B')
        
        if start_month == end_month:
            return start_month
        else:
            return f"{start_month}-{end_month}"

    summary['Month'] = summary['Неделя года'].apply(get_month_from_week)
    summary['Year'] = 2024  # или нужный год
    summary.to_excel(writer, sheet_name='Pivot', index=False)

    summary_by_category['Month'] = summary_by_category['Неделя года'].apply(get_month_from_week)
    summary_by_category['Year'] = 2024  # или нужный год
    summary_by_category.to_excel(writer, sheet_name='Category', index=False)

    # Для last_data_unique тоже только Year добавим, month_name уже есть
    last_data_unique['Year'] = pd.to_datetime(last_data_unique['marketCollected']).dt.year
    last_data_unique.to_excel(writer, sheet_name='Last', index=False)